# Практическая работа №1

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

### Загрузим данные для работы. 

In [2]:
df = pd.read_csv("Elections.csv",delimiter=';')
df.head()

,Region Index,Subjects of Russia,Unemployment rate. %,Income per capita. RUB,Average salary. RUB per month,Spendings per capita. RUB,State housing. %,Higher Education. %,Number of doctors per 10000 citizens,GRP per capita. RUB,Investment in total capital,Consumer Price Index (CPI),Average income / min. living wage,Average costs / min. living wage,Votes for United Russia. %
0,1,Adygea,43505.0,12236,12787.4,8361,0.6,4.0,37.9,93548.3,25834,110.1,42036,17168.0,61.002
1,2,Altai,43685.0,10926,12050.7,8139,43680.0,43694.0,48.7,107282.4,23048,108.2,33604,15707.0,37.174
2,3,Amur Oblast,43714.0,14064,21207.5,9406,2.0,43590.0,62.9,175927.8,95619,109.4,17199,23743.0,43.543
3,4,Arkhangelsk Oblast,43503.0,19310,22192.1,11688,43527.0,9.0,56.5,256728.4,63912,109.4,14305,43587.0,31.900
4,5,Astrakhan Oblast,43532.0,14641,16582.1,10651,43586.0,43472.0,68.2,131400.1,56338,109.8,20852,31778.0,60.170


Будем решать задачу регрессии: необходимо предсказать процент голосов за Единую Россию в каждом регионе по различным характеристикам уровня жизни в регионе.

## Шаг 1. 
Создайте матрицу X объект-признак и целевой вектор y ("Votes for United Russia. %")

In [3]:
y = np.array(df["Votes for United Russia. %"])
X = np.array(df.drop(columns = ['Subjects of Russia', 'Votes for United Russia. %']))

In [4]:
X

array([[1.0000e+00, 4.3505e+04, 1.2236e+04, ..., 1.1010e+02, 4.2036e+04,
        1.7168e+04],
       [2.0000e+00, 4.3685e+04, 1.0926e+04, ..., 1.0820e+02, 3.3604e+04,
        1.5707e+04],
       [3.0000e+00, 4.3714e+04, 1.4064e+04, ..., 1.0940e+02, 1.7199e+04,
        2.3743e+04],
       ...,
       [8.1000e+01, 4.3685e+04, 2.3024e+04, ..., 1.0600e+02, 4.3589e+04,
        2.0121e+04],
       [8.2000e+01, 4.3559e+04, 4.1865e+04, ..., 1.0820e+02, 1.3332e+04,
        2.3802e+04],
       [8.3000e+01, 4.3623e+04, 1.4548e+04, ..., 1.1050e+02, 2.0486e+04,
        2.4838e+04]])

In [5]:
y

array([61.002, 37.174, 43.543, 31.9  , 60.17 , 70.501, 51.162, 50.118,
       49.016, 38.271, 35.484, 33.4  , 50.048, 91.441, 48.113, 43.281,
       40.119, 90.963, 34.932, 81.911, 37.072, 66.098, 40.415, 45.254,
       89.841, 32.26 , 64.238, 34.898, 58.807, 30.743, 56.152, 36.697,
       44.406, 45.724, 33.543, 40.085, 41.045, 52.237, 91.616, 46.615,
       32.834, 32.022, 36.037, 44.559, 34.578, 33.835, 39.61 , 34.895,
       38.986, 56.302, 36.284, 32.987, 36.652, 53.335, 50.217, 39.791,
       39.366, 35.369, 64.888, 41.907, 32.707, 67.899, 36.227, 49.115,
       66.658, 77.833, 38.441, 37.514, 85.294, 61.32 , 62.211, 45.094,
       43.563, 38.136, 40.134, 41.009, 50.285, 99.478, 43.42 , 70.32 ,
       49.159, 71.679, 29.039])

## Шаг 2.
Оставьте в матрице X только столбцы 'Unemployment rate. %', 'Spendings per capita. RUB', 'Number of doctors per 10000 citizens', 'Average salary. RUB per month'

In [6]:
X = np.array(df[['Unemployment rate. %', 'Spendings per capita. RUB', 'Number of doctors per 10000 citizens', 'Average salary. RUB per month']])
X

array([[4.35050e+04, 8.36100e+03, 3.79000e+01, 1.27874e+04],
       [4.36850e+04, 8.13900e+03, 4.87000e+01, 1.20507e+04],
       [4.37140e+04, 9.40600e+03, 6.29000e+01, 2.12075e+04],
       [4.35030e+04, 1.16880e+04, 5.65000e+01, 2.21921e+04],
       [4.35320e+04, 1.06510e+04, 6.82000e+01, 1.65821e+04],
       [9.00000e+00, 1.41730e+04, 4.22000e+01, 1.63777e+04],
       [4.35010e+04, 1.04280e+04, 4.04000e+01, 1.59384e+04],
       [8.00000e+00, 9.66900e+03, 3.62000e+01, 1.23256e+04],
       [4.35650e+04, 9.57200e+03, 4.60000e+01, 1.79997e+04],
       [4.35020e+04, 7.90200e+03, 3.31000e+01, 1.44844e+04],
       [8.00000e+00, 1.06310e+04, 5.05000e+01, 1.48561e+04],
       [4.37150e+04, 8.56900e+03, 3.46000e+01, 1.85364e+04],
       [4.35920e+04, 9.82200e+03, 5.26000e+01, 1.43373e+04],
       [4.36910e+04, 1.14130e+04, 3.83000e+01, 1.02435e+04],
       [4.35640e+04, 9.37500e+03, 3.54000e+01, 1.97180e+04],
       [4.35660e+04, 9.08300e+03, 5.61000e+01, 1.86845e+04],
       [4.36230e+04, 7.5

## Шаг 3.
Разбейте данные на train и test (доля тестовых данных - 30%).

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Шаг 4.
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [8]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

pred_test = regressor.predict(X_test)
pred_train = regressor.predict(X_train)
print(pred_test)
print(pred_train)

[43.28097663 48.17257578 47.0513895  47.11550197 49.468678   44.80231522
 64.1199033  50.93132644 46.75828022 46.99749458 48.11896893 47.16572181
 46.60695039 46.25794197 45.39375392 45.20398978 45.23777506 48.98570057
 42.40572789 44.10475771 49.14121339 47.61455538 60.72262703 48.20251684
 48.20080829]
[46.57992057 51.54279014 48.08038689 49.14114989 31.85592045 46.21615027
 47.88275428 64.44769077 47.74307875 45.32776657 48.49855925 71.74881966
 48.01969479 46.51946174 44.09542399 48.92181371 47.21471741 71.55589538
 49.94099007 48.29864587 43.55041966 46.82520553 48.85879096 46.16498982
 47.11956851 44.17705266 47.08059116 45.88383236 49.01905975 48.09903369
 44.55310203 65.59489865 65.78058852 49.91816614 45.76605116 51.87293797
 44.03916946 42.09681293 49.46586924 58.17992958 49.02695113 39.69537346
 45.64015652 45.9479586  49.35277486 49.17275562 48.13522255 47.4177787
 70.46073635 49.31584969 48.54356784 85.64444172 50.2263668  46.78596346
 41.10278982 48.74200445 48.98850165 4

## Шаг 5.
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [9]:
print('R2_test: ', r2_score(y_test, pred_test), '| MSE_test:', mean_squared_error(y_test, pred_test))
print('R2_train: ',r2_score(y_train, pred_train), '| MSE_train:',mean_squared_error(y_train, pred_train))

R2_test:  -0.0982274620649568 | MSE_test: 157.75546668177134
R2_train:  0.22334242322916253 | MSE_train: 263.7824437868954


## Шаг 6.
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [10]:
cross_val_score(regressor, X, y, cv=5, scoring = 'r2').mean()

-0.2787409060316637

## Шаг 7.
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1. 

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [11]:
from sklearn.linear_model import Lasso

for a in np.arange(0.1, 1.1, 0.1):
    regressor2 = Lasso(alpha=a)
    regressor2.fit(X_train, y_train)

    y_pred2 = regressor2.predict(X_test)

    print('alpha={}'.format(a))
    print('Scores:', r2_score(y_test,y_pred2), mean_squared_error(y_test, y_pred2))
    print(regressor2.coef_,'\n')

alpha=0.1
Scores: -0.09822689832352771 157.75538570283456
[-0.00040957 -0.00107834  0.00023761  0.00027907] 

alpha=0.2
Scores: -0.09822633436476447 157.7553046926787
[-0.00040957 -0.00107833  0.00023761  0.00027906] 

alpha=0.30000000000000004
Scores: -0.09822577139814181 157.75522382503937
[-0.00040957 -0.00107831  0.00023761  0.00027905] 

alpha=0.4
Scores: -0.09822520629584419 157.7551426506199
[-0.00040957 -0.0010783   0.00023761  0.00027905] 

alpha=0.5
Scores: -0.09822464120717833 157.75506147815858
[-0.00040957 -0.00107829  0.00023761  0.00027904] 

alpha=0.6
Scores: -0.09822408007838779 157.7549808745156
[-0.00040957 -0.00107827  0.00023761  0.00027904] 

alpha=0.7000000000000001
Scores: -0.09822351501751392 157.75489970604647
[-0.00040957 -0.00107826  0.00023761  0.00027903] 

alpha=0.8
Scores: -0.09822294997027292 157.75481853953565
[-0.00040957 -0.00107825  0.00023761  0.00027903] 

alpha=0.9
Scores: -0.09822238493666391 157.75473737498297
[-0.00040957 -0.00107823  0.000237

## Шаг 8.
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [12]:
from sklearn.linear_model import LassoCV

n_alphas = 200
alphas = np.linspace(0.1, 1.1, n_alphas)

lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=42)
lasso_cv.fit(X, y)

LassoCV(alphas=array([0.1       , 0.10502513, 0.11005025, 0.11507538, 0.1201005 ,
       0.12512563, 0.13015075, 0.13517588, 0.14020101, 0.14522613,
       0.15025126, 0.15527638, 0.16030151, 0.16532663, 0.17035176,
       0.17537688, 0.18040201, 0.18542714, 0.19045226, 0.19547739,
       0.20050251, 0.20552764, 0.21055276, 0.21557789, 0.22060302,
       0.22562814, 0.23065327, 0.23567839, 0.24070352, 0.24572864,
       0.2507...
       0.92914573, 0.93417085, 0.93919598, 0.94422111, 0.94924623,
       0.95427136, 0.95929648, 0.96432161, 0.96934673, 0.97437186,
       0.97939698, 0.98442211, 0.98944724, 0.99447236, 0.99949749,
       1.00452261, 1.00954774, 1.01457286, 1.01959799, 1.02462312,
       1.02964824, 1.03467337, 1.03969849, 1.04472362, 1.04974874,
       1.05477387, 1.05979899, 1.06482412, 1.06984925, 1.07487437,
       1.0798995 , 1.08492462, 1.08994975, 1.09497487, 1.1       ]),
        cv=5, random_state=42)

In [13]:
lasso_cv.coef_

array([-0.00036284, -0.00095417,  0.00027341,  0.00022369])

In [14]:
lasso_cv.alpha_

1.1

In [15]:
regressor2 = Lasso(alpha=lasso_cv.alpha_)

regressor2.fit(X_train, y_train)

y_pred2 = regressor2.predict(X_test)

print('mse:', mean_squared_error(y_test, y_pred2))

mse: 157.75457609268216


## Шаг 9.

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [16]:
regressor2 = Lasso(alpha=lasso_cv.alpha_)

regressor2.fit(X_train, y_train)

y_pred2 = regressor2.predict(X_test)

print('Scores:', r2_score(y_test, y_pred2))

Scores: -0.09822126215686011


## Шаг 10.

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import linear_model
pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures(2))])
X_poly = pipe.fit_transform(X)
cross_val_score(regressor, X_poly, y, cv=5, scoring = 'r2').mean()

-178880.7470298739

## Шаг 11.
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [18]:
pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures(2))])
X_train = pipe.fit_transform(X_train)
X_test = pipe.fit_transform(X_test)

regressor.fit(X_train, y_train)
regressor.fit(X_test, y_test)

LinearRegression()

In [19]:
pred_train_poly = regressor.predict(X_train)

In [20]:
pred_test_poly = regressor.predict(X_test)
pred_test

array([43.28097663, 48.17257578, 47.0513895 , 47.11550197, 49.468678  ,
       44.80231522, 64.1199033 , 50.93132644, 46.75828022, 46.99749458,
       48.11896893, 47.16572181, 46.60695039, 46.25794197, 45.39375392,
       45.20398978, 45.23777506, 48.98570057, 42.40572789, 44.10475771,
       49.14121339, 47.61455538, 60.72262703, 48.20251684, 48.20080829])

In [21]:
print('R2_test: ', r2_score(y_test, pred_test_poly), '| MSE_test:', mean_squared_error(y_test, pred_test_poly))
print('R2_train: ',r2_score(y_train, pred_train_poly), '| MSE_train:',mean_squared_error(y_train, pred_train_poly))

R2_test:  0.5929493502441775 | MSE_test: 58.471006629726716
R2_train:  -2853.4645693809603 | MSE_train: 969484.702570007


## Сделайте выводы. Для этого ответьте на вопросы:

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

1. Нет, достаточно плохое. У модели большое MSE
2. Не особо, MSE не изменилось
3. Да, повлияло. Линейная регрессия делает слишком грубую апроксимацию, полиномы более аккуратные в этом плане, поэтому и получается более низкий MSE. 

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

Сделал улучшение на 0.13( если можно поставить конечно 1.3 балла....)

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

In [22]:
X = np.array(df[['Unemployment rate. %', 'Spendings per capita. RUB', 'Number of doctors per 10000 citizens', 'Average salary. RUB per month']])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures(2))])
X = pipe.fit_transform(X)
from sklearn.linear_model import RidgeCV

n_alphas = 200
alphas = np.linspace(0.1, 1.2, n_alphas)

ridge_cv = RidgeCV(alphas=alphas, cv=7)
ridge_cv.fit(X, y)

RidgeCV(alphas=array([0.1       , 0.10552764, 0.11105528, 0.11658291, 0.12211055,
       0.12763819, 0.13316583, 0.13869347, 0.14422111, 0.14974874,
       0.15527638, 0.16080402, 0.16633166, 0.1718593 , 0.17738693,
       0.18291457, 0.18844221, 0.19396985, 0.19949749, 0.20502513,
       0.21055276, 0.2160804 , 0.22160804, 0.22713568, 0.23266332,
       0.23819095, 0.24371859, 0.24924623, 0.25477387, 0.26030151,
       0.26582...
       1.0120603 , 1.01758794, 1.02311558, 1.02864322, 1.03417085,
       1.03969849, 1.04522613, 1.05075377, 1.05628141, 1.06180905,
       1.06733668, 1.07286432, 1.07839196, 1.0839196 , 1.08944724,
       1.09497487, 1.10050251, 1.10603015, 1.11155779, 1.11708543,
       1.12261307, 1.1281407 , 1.13366834, 1.13919598, 1.14472362,
       1.15025126, 1.15577889, 1.16130653, 1.16683417, 1.17236181,
       1.17788945, 1.18341709, 1.18894472, 1.19447236, 1.2       ]),
        cv=7)

In [29]:
from sklearn.linear_model import Ridge
regressor2 = Ridge(alpha=lasso_cv.alpha_)
regressor2.fit(X, y)
y_pred2 = regressor2.predict(X)

print('Score:', r2_score(y, y_pred2))

Score: 0.720820648396978
